# Week 17, Lecture 01 CodeAlong
- Coefficients & Feature Importance

## Lesson Objectives

- By the end of this lesson, students will be able to:
    - Extract feature names from sklearn v1.1 objects
    - Extract and visualize coefficients
    - Save models to a joblib file


# Business Problem

The World Health Organization is creating a task force to increase life expectancy around the world.  They want to to focus their efforts on areas where countries can change to help their people live longer.

They have tasked you with identifying which statistics about each county have the greatest impact on life expectancy and how those could change in increase the lifespan of the population.

### The Data

Data comes from the World Health Organization.  It describes demographic, health, and economic data from countries around the world between 2000 and 2015. 

Each row is one country during one year.

> Task Inspired by: https://medium.com/@shanzehhaji/using-a-linear-regression-model-to-predict-life-expectancy-de3aef66ac21

- Kaggle Dataset on Life Expectancy:
    - https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who

In [ ]:
## Our standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Preprocessing tools
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

## Models & evaluation metrics
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn import set_config
set_config(transform_output='pandas')

import joblib

## setting random state for reproducibility
SEED = 321
np.random.seed(SEED)
## Matplotlib style
fav_style = ('ggplot','tableau-colorblind10')
fav_context  ={'context':'notebook', 'font_scale':1.1}
plt.style.use(fav_style)
sns.set_context(**fav_context)
plt.rcParams['savefig.transparent'] = False
plt.rcParams['savefig.bbox'] = 'tight'

In [ ]:
df = pd.read_csv("Data/Life Expectancy Data.csv")
df.info()
df.head(3)

In [ ]:
def evaluate_regression(model, X_train,y_train, X_test, y_test,for_slides=True): 
    """Evaluates a scikit learn regression model using r-squared and RMSE
    FOR SLIDES VERS DOES MULTIPLE PRINT STATEMENTS FOR VERTICAL DISPLAY OF INFO"""
    
    ## Training Data
    y_pred_train = model.predict(X_train)
    r2_train = metrics.r2_score(y_train, y_pred_train)
    rmse_train = metrics.mean_squared_error(y_train, y_pred_train, 
                                            squared=False)
    mae_train = metrics.mean_absolute_error(y_train, y_pred_train)
    

    ## Test Data
    y_pred_test = model.predict(X_test)
    r2_test = metrics.r2_score(y_test, y_pred_test)
    rmse_test = metrics.mean_squared_error(y_test, y_pred_test, 
                                            squared=False)
    mae_test = metrics.mean_absolute_error(y_test, y_pred_test)
    
    if for_slides:
        df_version =[['Split','R^2','MAE','RMSE']]
        df_version.append(['Train',r2_train, mae_train, rmse_train])
        df_version.append(['Test',r2_test, mae_test, rmse_test])
        df_results = pd.DataFrame(df_version[1:], columns=df_version[0])
        df_results = df_results.round(2)
        display(df_results.style.hide(axis='index').format(precision=2, thousands=','))
        
    else: 
        print(f"Training Data:\tR^2 = {r2_train:,.2f}\tRMSE = {rmse_train:,.2f}\tMAE = {mae_train:,.2f}")
        print(f"Test Data:\tR^2 = {r2_test:,.2f}\tRMSE = {rmse_test:,.2f}\tMAE = {mae_test:,.2f}")



In [ ]:
# clean extra spaces
df.columns = df.columns.str.strip()
df

In [ ]:
pd.set_option('display.max_columns',0)
df

## EDA

In [ ]:
df.isna().sum()

> Can't have null values for the target!

In [ ]:
# drop null values ONLY FROM TARGET
df = df.dropna(subset=['Life expectancy'])

In [ ]:
df.describe()

In [ ]:
target = 'Life expectancy'

grid_spec = {'height_ratios':[0.8,0.2]}
fig, axes = plt.subplots(nrows=2, figsize=(6,5), gridspec_kw=grid_spec)

sns.histplot(data=df, x=target,ax=axes[0])
sns.boxplot(data=df, x=target, ax=axes[1]);

## Preprocessing (with Sklearn v1.1+)

In [ ]:
# ### Train Test Split
## Make x and y variables
target = "Life expectancy"
drop_feats = []

y = df[target].copy()
X = df.drop(columns=[target, *drop_feats]).copy()

## train-test-split with random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=SEED)
X_train.head(3)

### Drop Country Columns

We are going to drop the country columns.  Why?  3 reasons:

1. The ultimate goal of the business problem is to focus on high impact areas of change.  A country cannot change what it is.  Zimbabwe cannot become Sweden!  Instead, we will focus on the features that CAN be changed.

2. After one-hot encoding there are too many features to analyze.  We need to reduce our focus

3. one-hot encoded countries add multi-collinearity to our features, reducing the reliability of our analysis.

In [ ]:
X_train = X_train.drop(columns='Country')
X_test = X_test.drop(columns='Country')

### Categorical Processing Pipeline

In [ ]:
## Make numeric preprocessing pipeline
num_sel = make_column_selector(dtype_include='number')
mean_imputer = SimpleImputer(strategy='mean')

num_tuple = ('Numeric', mean_imputer, num_sel)

### Numeric Processing Pipeline

<center> <font color='red' size=5>Notice We Are Not Scaling!!! </font>
    
   **<center> Q: Why not? </center>**

In [ ]:
## Make categorical preprocessing pipeline
## Drop one of the binary columns after OHE to reduce multicollinearity
cat_sel = make_column_selector(dtype_include='object')

cat_pipe = make_pipeline(SimpleImputer(strategy='constant',
                                       fill_value='MISSING'),
                         OneHotEncoder(handle_unknown='ignore', 
                                       sparse_output=False,
                                       drop='if_binary'))

cat_tuple = ('Categorical', cat_pipe, cat_sel)

In [ ]:
## make the preprocessing column transformer
preprocessor = ColumnTransformer([num_tuple, cat_tuple],
                                verbose_feature_names_out=False)
preprocessor

### Creating Unscaled Processed df

In [ ]:
## fit preprocessor and transform data
preprocessor.fit(X_train)
X_train_proc = preprocessor.transform(X_train)
X_test_proc = preprocessor.transform(X_test)
X_train_proc.head()

# Modeling - Linear Regression

### Linear Model Assumptions

**Linearity:**
That the input features have a linear relationship with the target.

**Independence of Features:** (AKA Little-to-No Multicollinearity)
That the features are not strongly related to other features.

**Normality:**
The model's residuals are approximately normally distributed.

**Homoscedasticity:**
The model residuals have equal variance across all predictions.

## Model 1: LinReg

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train_proc, y_train)
evaluate_regression(lin_reg, X_train_proc, y_train, 
                    X_test_proc, y_test)

### Extracting and Visualizing Coefficients

#### Extracting Coefficients from lin_reg

In [ ]:
# access the .coef_ 



In [ ]:
# Intercept



In [ ]:
## Saving the coefficients



In [ ]:
## Add the Intercept



### def `get_coefficients`

In [ ]:
## formatting numbers to not use , thousands sep, and 4 digits floats



In [ ]:
## show coefs again



In [ ]:
# Define get_coefficients function to extract LinReg coefficients



## Visualizing Coefficients

In [ ]:
## Plot Coefficients



### Drop the Intercept

It's hard to see the other values because the intercept is so large

In [ ]:
## Visualize Coefficients without Intercept



In [ ]:
## Function for plotting coefficients

def plot_coefficients(coefs):
    coefs = coefs.drop('intercept')
    ax = coefs.sort_values().plot(kind='barh', figsize=(5,5))
    ax.axvline(0, color='k')
    ax.set(xlabel='Life Expectancy', title="Coefficients - Countries")
    plt.show();

> ***Q1: What do we notice about our coefficients? Is there anything odd that would be difficult to a stakeholder?***

> ***Q2: What does that intercept represent?***

> ***Q3: What would it mean if we did not use an intercept? (fit_intercept=False)***

# Understanding Coefficients

In [ ]:
## Display Coefficients

## Plot sorted Coefficients



Why does GDP per Capita seem to have no effect on Life Expectancy?

In [ ]:
# Median GDP



The median GDP per Capita in the dataset is $3,184 US.

In [ ]:
# GDP Coefficient



GDP per Capita adds 1 year of life for every 44,500 US Dollars of GDP per Capita.  

In [ ]:
## Effect of median GDP on target



Countries with the median average GDP per Capita add about 2 months of expected life from this feature.

In [ ]:
## Max GDP



In [ ]:
# Effect: Max GDP times coefficient



The country with the highest GDP per Capita adds 5 years to life expectancy with this feature.

In [ ]:
# What country is that?




What about Population?

In [ ]:
# Population coefficient



In [ ]:
# Population median



In [ ]:
# effect of coefficient on median population country



The countries with a median average population add less than a day to their life expectancy.

However...

In [ ]:
# Max population



In [ ]:
# Effect of max population on target



The country with the highest population adds 5 years as a result of the population size!

In [ ]:
## What country is that?



## Feature Importance

**Feature importance does <span style="color:red"> NOT </span> describe the relationship between features targets.**

**It only describes what the model is focusing on to make its predictions.**

**Think of the results as percentage weights on the features for how important they are.**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train_proc, y_train)
evaluate_regression(rf_reg, X_train_proc, y_train, 
                    X_test_proc, y_test)

> Using the models .feature_names_in_

In [ ]:
# Extract Feature Importances




In [ ]:
# create a function to extract importances




In [ ]:
# show importances Series



In [ ]:
# Plot Importances




> **Q1:** What do these numbers mean?

> **Q2:** What are the top 5 most important features?

## Using joblib to Save our Model, Data, and Objects

In [ ]:
X_train.head()

In [ ]:
import joblib, os

## creating a dictionary of all of the variables to save for later




In [ ]:
# Create the folder to save it in



In [ ]:
# Save the models, data, and preprocessor




In [ ]:
# try loading again to make sure it works.




> We will continue working with this task and these models next class!

# *Teaser* Shap (For Regression)

In [ ]:
# Import and init shap
import shap
shap.initjs()

<font color='red'><center>We MUST preprocess the data ahead of time for SHAP</center></font>

In [ ]:
# Take a sample of the training data
X_shap = shap.sample(X_train_proc, nsamples = 500, random_state=SEED)
y_shap = y_train.loc[X_shap.index]

# Instantiate a Model Explainer with the model
explainer = shap.Explainer(rf_reg)

## Get shap values form the explainer
shap_values = explainer(X_shap,y_shap)

In [ ]:
shap.summary_plot(shap_values, features = X_shap)